In [54]:
import pymongo
from pymongo import MongoClient

import numpy as np
import pandas as pd
import pickle
from pprint import pprint
from IPython.display import clear_output

In [55]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["github_projects"]

In [4]:
!mongorestore -d github_usernames github_usernames_dump  #  clean_ready_users

db.list_collection_names()

2019-03-22T12:14:13.687-0700	the --db and --collection args should only be used when restoring from a BSON file. Other uses are deprecated and will not exist in the future; use --nsInclude instead
2019-03-22T12:14:13.687-0700	building a list of collections to restore from github_usernames_dump dir
2019-03-22T12:14:13.688-0700	don't know what to do with file "github_usernames_dump/.DS_Store", skipping...
2019-03-22T12:14:13.688-0700	don't know what to do with subdirectory "github_usernames_dump/github_usernames", skipping...
2019-03-22T12:14:13.688-0700	done


['users']

In [56]:
collection = db.get_collection('users')
cursor = db.users.find({ "following":{"$exists":True }, "followers":{"$exists" : True}}) # get users who have both following list and followers

In [57]:
count = cursor.count()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


In [58]:
print(count)

50295


In [59]:
# #cursor = collection.find({})
# for document in cursor: 
#     print(document['username']) #document['following'])# (

In [9]:
!mongodump -d github_usernames -o clean_ready_users

2019-03-22T12:14:15.893-0700	writing github_usernames.users to 
2019-03-22T12:14:15.965-0700	done dumping github_usernames.users (11809 documents)


In [60]:
df1 = pd.DataFrame(list(cursor))

In [61]:
#pprint(df1.head())

In [62]:
df2 = df1[['username','following']].copy()
df2['rating'] = 1

In [63]:
df2.head(20)

,username,following,rating
0,donnemartin,"[mitsuhiko, kenneth-reitz, wesm, jakevdp]",1
1,kennethreitz,[],1
2,google,[],1
3,tensorflow,[],1
4,pallets,[],1
5,django,[],1
6,mitsuhiko,"[brosner, alex, ericflo, bryanveloso, jezdez, ...",1
7,jakubroztocil,"[mojombo, defunkt, roncohen, alex, jashkenas, ...",1
8,ansible,[],1
9,nvbn,"[vgarvardt, hdhog, dtantsur, zcho, chemikadze,...",1


In [64]:
df3 = pd.DataFrame(df2.following.tolist(), index=df2.username).stack().reset_index(level=1, drop=False) # stacking by username with followers
df3.head()

,level_1,0
username,,
donnemartin,0,mitsuhiko
donnemartin,1,kenneth-reitz
donnemartin,2,wesm
donnemartin,3,jakevdp
mitsuhiko,0,brosner


In [65]:
df3.rename({0:"following"},axis=1, inplace=True)

In [66]:
df3.rename({"level_1":"rating"},axis=1, inplace=True)

In [67]:
df3['rating']=1
df3.head()

,rating,following
username,,
donnemartin,1,mitsuhiko
donnemartin,1,kenneth-reitz
donnemartin,1,wesm
donnemartin,1,jakevdp
mitsuhiko,1,brosner


In [68]:
df4 = df3.reset_index(level=0, drop=False)

In [69]:
df4.head()

,username,rating,following
0,donnemartin,1,mitsuhiko
1,donnemartin,1,kenneth-reitz
2,donnemartin,1,wesm
3,donnemartin,1,jakevdp
4,mitsuhiko,1,brosner


In [20]:
with open("data/df4.pickle", 'wb') as fp:
    pickle.dump(df4, fp)

# shape to % of data, (move to aws machine to) run surprise

In [70]:
new_df = df2
# new_df = new_df.replace(to_replace='', value=np.nan).dropna() # works when space delimeted string in followers col
new_df = new_df[new_df.astype(str)['following'] != '[]'] # works on strings in list
new_df.head(20)
#new_df.shape

,username,following,rating
0,donnemartin,"[mitsuhiko, kenneth-reitz, wesm, jakevdp]",1
6,mitsuhiko,"[brosner, alex, ericflo, bryanveloso, jezdez, ...",1
7,jakubroztocil,"[mojombo, defunkt, roncohen, alex, jashkenas, ...",1
9,nvbn,"[vgarvardt, hdhog, dtantsur, zcho, chemikadze,...",1
15,josephmisiti,"[mojombo, defunkt, vanpelt, macournoyer, ry, b...",1
19,jkbrzt,[jakubroztocil],1
21,minimaxir,[Stitchpunk],1
35,eriklindernoren,"[karpathy, alexjc, fchollet, llSourcell, junya...",1
36,wangshub,"[kenneth-reitz, paulyang1990, dongweiming, lil...",1
39,soimort,"[josh, maccman, jgm, aaronsw, sethladd, philc,...",1


In [71]:
# create col to sort by "num following" to get most interesting users
new_df['follow_num'] = new_df['following'].str.len() #len(text.split(','))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [72]:
new_df.head(20)

,username,following,rating,follow_num
0,donnemartin,"[mitsuhiko, kenneth-reitz, wesm, jakevdp]",1,4
6,mitsuhiko,"[brosner, alex, ericflo, bryanveloso, jezdez, ...",1,22
7,jakubroztocil,"[mojombo, defunkt, roncohen, alex, jashkenas, ...",1,30
9,nvbn,"[vgarvardt, hdhog, dtantsur, zcho, chemikadze,...",1,9
15,josephmisiti,"[mojombo, defunkt, vanpelt, macournoyer, ry, b...",1,30
19,jkbrzt,[jakubroztocil],1,1
21,minimaxir,[Stitchpunk],1,1
35,eriklindernoren,"[karpathy, alexjc, fchollet, llSourcell, junya...",1,11
36,wangshub,"[kenneth-reitz, paulyang1990, dongweiming, lil...",1,27
39,soimort,"[josh, maccman, jgm, aaronsw, sethladd, philc,...",1,30


In [104]:
new_df = new_df.sort_values('follow_num', ascending=False)

# Create sample of top users to find out if surprise works at all
top_1000 = new_df.head(1000)
top_1000.head()

,username,following,rating,follow_num
50294,nishowsan,"[josh, benschwarz, andrew, tmm1, mitsuhiko, da...",1,30
22223,enricofer,"[mourner, yanokwa, sgillies, tschaub, 3nids, c...",1,30
33574,srynot4sale,"[ry, chilts, aaronsw, nikz, mitsuhiko, aaronha...",1,30
18342,riywo,"[mojombo, wycats, macournoyer, mattetti, al3x,...",1,30
18343,ryanj,"[bryanl, jchris, ncdc, mikeal, quirkey, krober...",1,30


In [91]:
# stack dataframes

# stacked_new_df = pd.DataFrame(new_df.following.tolist(), index=new_df.username).stack().reset_index(level=1, drop=False)
# stacked_new_df.head()

stacked_top_1000 = pd.DataFrame(top_1000.following.tolist(), index=top_1000.username).stack().reset_index(level=1, drop=False)
stacked_top_1000.head()

,level_1,0
username,,
nishowsan,0,josh
nishowsan,1,benschwarz
nishowsan,2,andrew
nishowsan,3,tmm1
nishowsan,4,mitsuhiko


In [ ]:
# get 0 followed scores for (not followed) col values for each primary user who has followed people

# edit more to get all unfollowed people from col "followed", add with 0 rating

stacked_1k_zeroed = pd.DataFrame()

for username in stacked_top_1000['username'].unique():
    #print(username)
    stacked_new_df2 = stacked_new_df2.append({'username': username, 'rating': int(0), 'following': 'alexpfox'}, ignore_index=True)
    

In [109]:
# from rec workbook
# if these don't run, check cells below to get stacked_top_1000 into memory

def get_following(username):
    following = set(stacked_top_1000.loc[stacked_top_1000['username'] == username, 'following']) 
    return following

def get_all_following(): # take from stacked list so they're separated nicely
    return set(stacked_top_1000['following'].unique())

def get_not_following(username):

    following = get_following(username)  # get list username _is_ following
    all_follows = get_all_following() # get list everyone has followed, all possible from following col
    not_following = list(all_follows - following) # difference is list of people username is not following
        
    return not_following

print(get_not_following('nishowsan'))

['ferki', 'kachayev', 'baijum', 'stefan2904', 'tomcart', 'jace', 'lsegal', 'bittner', 'eyedol', 'jacobandreas', 'aprilzero', 'nelix', 'hit-computer', 'mavam', 'pierre-24', 'lidaobing', 'MTrop', 'borisjoffe', 'jaydeep', 'RncLsn', 'Neo23x0', 'gabeio', 'ylecun', 'andremw', 'aaronbieber', 'tiancaiamao', 'rmit-s3511441-rashivromio-bhusal', 'mattygio77', 'thejefflarson', 'dhafinmuktafin', 'ftrain', 'Altazimuth', 'oesteban', 'chenxiaolong', 'waynegraham', 'philsong', 'davidgao', 'mikemol', 'ivanma72', 'jone', 'DylanLenz', 'soltysh', 'yasnari', 'hberntsen', 'nicksieger', 'aglee', 'jblomo', 'evandrix', 'rajat503', 'ExTEnS10N', 'superisaac', 'mncoppola', 'apkelly', 'dplord', 'jinq0123', 'clh021', 'imgntn', 'mjescobar', 'ysmood', 'stevetranby', 'fjsj', 'alansaid', 'doochik', 'des4maisons', 'joshuaskelly', 'andreas-g', 'gvishal', 'carpedm20', 'tewilove', 'jstorimer', 'benanne', 'ydye', 'baldur', 'ustbhuangyi', 'ghostrong', 'bangpound', 'smallnest', 'leixiaohua1020', 'renard', 'braddunbar', 'dnouri

In [120]:
def create_not_following(user, no_follows, limit):
    
    temp_df = pd.DataFrame() #accum
    
    for no_follow in no_follows:
        if limit <=0: # get only limit # of 0 val followed
            break
        temp_df = temp_df.append({'username': user, 'rating': int(0), 'following': no_follow}, ignore_index=True)
        limit -=1
        
    return temp_df

In [ ]:
# test code for create_not_following
usr = 'nishowsan'
create_not_following(usr, get_not_following(usr), limit=100)

In [124]:
# add not following list(0 val follow entries) to stacked_top_1000

# create accumulator df
thou_zero_vals = pd.DataFrame()

#loop through 1k list of users to add rows for '0 followed users' in long list for each 1k user

for username in top_1000['username']:
    no_follows = get_not_following(username)
    
    thou_zero_vals = thou_zero_vals.append(create_not_following(username, no_follows, limit=100))

In [125]:
print(thou_zero_vals)

                           following  rating   username
0                              ferki     0.0  nishowsan
1                           kachayev     0.0  nishowsan
2                             baijum     0.0  nishowsan
3                         stefan2904     0.0  nishowsan
4                            tomcart     0.0  nishowsan
5                               jace     0.0  nishowsan
6                             lsegal     0.0  nishowsan
7                            bittner     0.0  nishowsan
8                             eyedol     0.0  nishowsan
9                       jacobandreas     0.0  nishowsan
10                         aprilzero     0.0  nishowsan
11                             nelix     0.0  nishowsan
12                      hit-computer     0.0  nishowsan
13                             mavam     0.0  nishowsan
14                         pierre-24     0.0  nishowsan
15                         lidaobing     0.0  nishowsan
16                             MTrop     0.0  ni

In [219]:
full_df = stacked_top_1000.append(thou_zero_vals)
full_df.sample(100)

,following,rating,username
77,smallnest,0.0,marcellmars
19754,cloudwu,1.0,wangyves
18,jaydeep,0.0,youzaka
86,ubermuda,0.0,cj1324
37,mikemol,0.0,jone
12028,fqrouter,1.0,qwqmeow
0,ferki,0.0,jelly
70,jstorimer,0.0,vincentdchan
0,ferki,0.0,sxjscience
35,philsong,0.0,nondanee


In [220]:
# type cast rating to int
full_df['rating'] = full_df['rating'].astype(int)
full_df.head()

,following,rating,username
0,josh,1,nishowsan
1,benschwarz,1,nishowsan
2,andrew,1,nishowsan
3,tmm1,1,nishowsan
4,mitsuhiko,1,nishowsan


In [130]:
# export full_df to use in predictions on aws
with open("data/0d_1k_df.pickle", 'wb') as fp:
    pickle.dump(full_df, fp)

In [132]:
full_df.shape

(130000, 3)

In [221]:
full_df

,following,rating,username
0,josh,1,nishowsan
1,benschwarz,1,nishowsan
2,andrew,1,nishowsan
3,tmm1,1,nishowsan
4,mitsuhiko,1,nishowsan
5,davetron5000,1,nishowsan
6,tj,1,nishowsan
7,BanzaiMan,1,nishowsan
8,weierophinney,1,nishowsan
9,rkh,1,nishowsan


In [222]:
# create fake db from full_df in order to present something substantial
full_df_fake = full_df
full_df_fake.sample(40)

,following,rating,username
27727,mxcl,1,kid1412z
60,stevetranby,0,qwedc001
26766,somebee,1,peterhil
59,ysmood,0,ihebski
64,des4maisons,0,kevgathuku
10175,nsf,1,honix
91,AdamKinney,0,bkeating
8,eyedol,0,mastergreg
17,borisjoffe,0,kylefox
17068,simonpj,1,fuuzetsu


In [223]:
v = full_df_fake.rating.values == 0 # randomly assign 1 on top of 0's with probability p
full_df_fake.loc[v, 'rating'] = np.random.choice((0, 1), v.sum(), p=(.4, .6))

In [224]:
full_df_fake.sample(100)

,following,rating,username
42,yasnari,1,samyk
28433,geralt-encore,1,mikhaildubov
69,tewilove,1,trhura
1,kachayev,0,gustavohenrique
36,davidgao,1,sundaysec
48,rajat503,1,thinxer
85,jh2oman,1,deavmi
1,kachayev,0,dimazest
27,mattygio77,1,mr-karan
15848,stuarthalloway,1,sunng87


In [225]:
# export full_df_fake to use in predictions on aws for presentation
with open("data/fake_data.pickle", 'wb') as fp:
    pickle.dump(full_df_fake, fp)

In [107]:
# fix axis names
# stacked_new_df.rename({0:"following"},axis=1, inplace=True)
# stacked_new_df.rename({"level_1":"rating"},axis=1, inplace=True)

stacked_top_1000.rename({0:"following"},axis=1, inplace=True)
stacked_top_1000.rename({"level_1":"rating"},axis=1, inplace=True)

In [93]:
# stacked_new_df.head()

stacked_top_1000.head()

,rating,following
username,,
nishowsan,0,josh
nishowsan,1,benschwarz
nishowsan,2,andrew
nishowsan,3,tmm1
nishowsan,4,mitsuhiko


In [97]:
# fix levels after stack
# stacked_new_df = stacked_new_df.reset_index(level=0, drop=False)

stacked_top_1000= stacked_top_1000.reset_index(level=0, drop=False)
stacked_top_1000.head()

,level_0,index,username,rating,following
0,0,0,nishowsan,0,josh
1,1,1,nishowsan,1,benschwarz
2,2,2,nishowsan,2,andrew
3,3,3,nishowsan,3,tmm1
4,4,4,nishowsan,4,mitsuhiko


In [98]:
# stacked_new_df.drop('index', inplace=True, axis = 1)

stacked_top_1000.drop('index', inplace=True, axis = 1)

In [99]:
stacked_new_df.head()

stacked_top_1000.head()

,level_0,username,rating,following
0,0,nishowsan,0,josh
1,1,nishowsan,1,benschwarz
2,2,nishowsan,2,andrew
3,3,nishowsan,3,tmm1
4,4,nishowsan,4,mitsuhiko


In [101]:
# reset rating col, remove level col

#stacked_new_df['rating']=1
#stacked_new_df.drop('level_0', inplace=True, axis = 1)
#stacked_new_df.head()

stacked_top_1000['rating']=1
stacked_top_1000.drop('level_0', inplace=True, axis = 1)
stacked_top_1000.head()

# now stacked_new_df is a clean stacked working copy ordered by number of people user is following


,username,rating,following
0,nishowsan,1,josh
1,nishowsan,1,benschwarz
2,nishowsan,1,andrew
3,nishowsan,1,tmm1
4,nishowsan,1,mitsuhiko


In [103]:
len(stacked_new_df['username'].unique())

len(stacked_top_1000['username'].unique()) # 3000 entries, 1k unique names

30000

In [85]:
stacked_new_df.shape

(435472, 3)

In [86]:
# add in negative rating info

stacked_new_df2 = pd.DataFrame()

for username in stacked_new_df['username'].unique():
    #print(username)
    stacked_new_df2 = stacked_new_df2.append({'username': username, 'rating': int(0), 'following': 'alexpfox'}, ignore_index=True)
    
# for username in stacked_new_df2['username'].unique():
#     # print(username)
#     # if they do actually follow josh, don't add him
#         stacked_new_df2 = stacked_new_df2.append({'username':username, 'rating':0, 'following': 'alexpfox'}, ignore_index=True)
#         continue
#     else:
#         #stacked_new_df2 = stacked_new_df2.append({'username':username, 'rating':0, 'following': 'josh'}, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
stacked_zeros_df = pd.DataFrame()
stacked_zeros_df.shape

In [ ]:
stacked_zeros_df = stacked_new_df.append(stacked_new_df2)

stacked_zeros_df.head()

In [ ]:
stacked_zeros_df.shape

In [ ]:
#stacked_new_df2.loc[stacked_new_df2['rating'] == 0] 

In [51]:
with open("data/stacked_zeros_df.pickle", 'wb') as fp:
    pickle.dump(stacked_zeros_df, fp)

In [52]:
stacked_zeros_df.shape

(466200, 3)

## reshape df

In [ ]:
df2.head()

In [ ]:
df5 = df2

# print(df5.head)

for username in range(0, len(df2)):
    df5.at[username, 'following'] = ' '.join(df5.at[username, 'following'])
    
df5.head(20)

In [ ]:
df5.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
cv = CountVectorizer()

In [ ]:
cv1 = cv.fit_transform(df2.following) # creates dense matrix

In [ ]:
print(cv1)

In [ ]:
cv2 = cv1.toarray()

In [ ]:
print(cv2)

In [ ]:
with open("data/df.pickle", 'wb') as fp:
    pickle.dump(df5, fp)

## Build a recommender

In [ ]:
# happening in aws

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

#dfs['rating'] = 5
#df.head()

reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = cv2 #Dataset.load_from_df(dfs[['username', 'following', 'rating']], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)